In [ ]:
# To see interactive 3d display of atom locations 
# Instructions: 1) Set picfile to the location of the case you want to see, then run (or play) this top cell
#                  Run this only once when changing the input file
#               2) In second cell, choose units of center ('cell', or 'angstroms'), the center in nonrotated coordinates,
#                  and the width in angstroms.
#                  Run this cell for each change in center & width parameters
#               3) In the thrid cell, set the displacement_cut (units of cell lengths)
#                  Run this cell for each change in displacement
#                  Size and color sliders are for normal and displaced atoms
# Interaction includes size And color of normal and displaced atoms, zoom, rotate, stereoscopic view option, and fullscreen option
# The .xyz file is only needed. (To see an optional listing of vacancie and interstial sites- set print_void_fills to True.)
# The default rotation option was PKA, which mean that the PKA (big black sphere) should be at lower Z, somewhat centered on X,Y
# The default colors are black for the interstitials and white for the vacanices
# A larger black dot is used to indicate the location of the PKA hit
# Units are in cell lengths, all xyz scales are the same (e.g., 50x50x50)

base_path = "/users/lepoire/downloads/"
filename = "003-fpos-150-107.xyz"
print_void_fills=False # set to True to see locations of all vacancies and interstitials

import os, numpy as np, pickle, math
casestats=[]
path_to_file = os.path.join(base_path, filename)
fd = open(path_to_file , 'r')
print ('File:', filename)
if print_void_fills:
    file_void_fills=filename.replace(".xyz",".picklenorot")
    path_to_file = os.path.join(base_path, file_void_fills)
    with open(path_to_file, 'rb') as f:
        data = pickle.load(f)
    print (path_to_file)
    print (data)

# Read data file
header1 = fd.readline()
header2 = fd.readline()
header2=header2.strip()
columns=header2.split()
length=float(columns[7])
if float(filename[0:3])>4.5:
    celllength=2.8699
else:
    celllength=3.1652 #2.8699 #2.855294 #for 100 #2.86 in doc #2.8699 good for everything except E=100
data=[]
side= (length-celllength)/2. 
for line in fd:
    line = line.strip()
    columns = line.split()
    source = {}
    source['xpos'] = float(columns[1])
    source['ypos'] = float(columns[2])
    source['zpos'] = float(columns[3])
    source['xind'] = math.floor((source['xpos']+side)/celllength+0.5)  
    source['yind'] = math.floor((source['ypos']+side)/celllength+0.5)
    source['zind'] = math.floor((source['zpos']+side)/celllength+0.5)
    data.append(source)
m=len(data)
fd.close()

In [ ]:
# Second cell: Choose units of center ('cell', or 'angstroms'), the center in nonrotated coordinates,and the width in angstroms.
#              Run this for each change in center & width parameters#select atoms to plot

center_units='cell'  # cell lengths from edge or otherwise coordiante system of xyz file
center=[73,70,67] #[16.333,21.3333,27.7] #absolute:[-42.,3.,3.][-100.,-100.,-43][-120,-120,-63] #[-100.,-100.,-43]; cell:[16.333,21.3333,27.7]
width=50 # in Angstroms

if center_units=='cell': # cell lengths from edge, same as results from Fe_all output
    center[0]=center[0]*celllength-length/2.
    center[1]=center[1]*celllength-length/2.
    center[2]=center[2]*celllength-length/2.
# else: # absolute non-rotated center similar to coordinates in .xyz file
    
numatoms=0  # Just get a count of the number of atoms we'll be working with
for atom in data:
    xtest=abs((atom['xpos']-center[0])/width)
    if xtest<=0.5:
        ytest=abs((atom['ypos']-center[1])/width)
        if ytest<=0.5:
            ztest=abs((atom['zpos']-center[2])/width)
            if ztest<=0.5:
                numatoms+=1
# now get the locations and displacements                
x, y, z, c, selected = np.random.random((5, numatoms))
n2=math.floor((numatoms/2.)**(1/3))
n1=math.floor(n2/2.+0.5)-1
n3=n2+1
V = np.zeros((n3,n3,n3))
indx,nx,ny,nz=(0,)*4
for atom in data:
    xtest=abs((atom['xpos']-center[0])/width)
    if xtest<=0.5:
        ytest=abs((atom['ypos']-center[1])/width)
        if ytest<=0.5:
            ztest=abs((atom['zpos']-center[2])/width)
            if ztest<=0.5:
                #print (atom)
                displx1=(atom['xpos']+0.25*celllength)/celllength  # get displacements from corner(1) and center(2) locations
                disply1=(atom['ypos']+0.25*celllength)/celllength
                displz1=(atom['zpos']+0.25*celllength)/celllength
                displx2=(atom['xpos']+0.75*celllength)/celllength
                disply2=(atom['ypos']+0.75*celllength)/celllength
                displz2=(atom['zpos']+0.75*celllength)/celllength
                delx1=displx1-math.floor(displx1+0.5)
                dely1=disply1-math.floor(disply1+0.5)
                delz1=displz1-math.floor(displz1+0.5)
                delx2=displx2-math.floor(displx2+0.5)
                dely2=disply2-math.floor(disply2+0.5)
                delz2=displz2-math.floor(displz2+0.5)
                del1=(delx1*delx1+dely1*dely1+delz1*delz1)**0.5
                del2=(delx2*delx2+dely2*dely2+delz2*delz2)**0.5
                x[indx]=atom['xpos']-center[0]
                y[indx]=atom['ypos']-center[1]
                z[indx]=atom['zpos']-center[2]
                if del1>del2:  # find smallest displacement from a valid lcoation
                    displ=del2
                else:
                    displ=del1
                inx=math.floor(x[indx]/celllength+0.5)+n1
                iny=math.floor(y[indx]/celllength+0.5)+n1
                inz=math.floor(z[indx]/celllength+0.5)+n1
                if displ>V[inz,iny,inx]:  # if more than one at this location choose largest
                    V[inz,iny,inx]=displ
                c[indx]=displ
                indx+=1             
                
               

In [ ]:
# In the third cell, set the displacement_cut (units of cell lengths)
# Run this cell for each change in displacement
# Size and color sliders are for normal and displaced atoms
import ipyvolume as ipv, numpy as np
displacement_cut=0.25
indx=-1
for xs in c:
    indx+=1
    if xs > displacement_cut:
        selected[indx]=indx
    else:
        selected[indx]=0
colors = [(1.,1.,1.) if x2 <= displacement_cut else (1., 0., 0.) for x2 in c]
ipv.figure()
quiver = ipv.scatter(x, y, z, size=1, size_selected=2,selected=selected, color='white', color_selected='red', marker="sphere")

from ipywidgets import FloatSlider, ColorPicker, VBox, jslink
size = FloatSlider(min=0, max=3, step=0.1)
size_selected = FloatSlider(min=0, max=3, step=0.1)
color = ColorPicker()
color_selected = ColorPicker()
jslink((quiver, 'size'), (size, 'value'))
jslink((quiver, 'size_selected'), (size_selected, 'value'))
jslink((quiver, 'color'), (color, 'value'))
jslink((quiver, 'color_selected'), (color_selected, 'value'))
VBox([ipv.gcc(), size, size_selected, color, color_selected])

In [ ]:
# In this fourth cell, the displacement value is color coded
# Run this cell for each change in in cells 1 & 2 to see updated results
# Opacity and displacement level sliders are for the three colors RGB
import numpy as np
import ipyvolume as ipv
ipv.quickvolshow(V, level=[.31, .10,0.08], opacity=[.15,.03,.030], level_width=0.1, data_min=0, data_max=1)